In [1]:
import sys
sys.path.append("/home/ahow/MyGitHub/oppai_challenge/utils")

from logs import print_args
from default_spark import spark
from spark_delta import merge, table_exists

23/11/16 15:19:38 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/11/16 15:19:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ahow/virtual_envs/ahow/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-26224313-863d-4254-8d98-f96f79cf8fef;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 99ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0 

In [2]:
class UsersBronzeETL:
    def __init__(self, spark, dt_start, dt_end, pk=None):
        self.spark = spark
        self.dt_start = dt_start
        self.dt_end = dt_end
        self.pk = self._get_default_pk(pk)
        self.default_date = '1901-01-01'
        
    @staticmethod
    def _get_default_pk(pk):
        if pk is None:
            return ['id_oid']
        return pk
        
    @print_args(print_kwargs=['source_tb'])
    def extract(self, source_tb):
        latest_update = f"COALESCE(updated_at_date, created_at_date, TIMESTAMP '{self.default_date}')"
        df = spark.sql(f"""
        WITH latest AS (
            SELECT 
                {','.join(self.pk)}
                , MAX({latest_update}) AS latest_update
            FROM {source_tb}
            WHERE {latest_update} BETWEEN '{self.dt_start}' AND '{self.dt_end}'
            GROUP BY {','.join(self.pk)}
        )
        SELECT s.* FROM {source_tb} s
        JOIN latest l
            ON {" AND ".join([f"s.{c}=l.{c}" for c in self.pk])}
            AND {latest_update}=l.latest_update
        WHERE {latest_update} BETWEEN '{self.dt_start}' AND '{self.dt_end}'
        """)
        return df
    
    def transform(self):
        pass
    
    @print_args(print_kwargs=['target_tb', 'drop'])
    def load(self, df, target_tb, drop=False):
        if drop and table_exists(target_tb, self.spark):
            print(f"Dropping table {target_tb}")
            self.spark.sql(f"DROP TABLE {target_tb}").show()

        merge(df, target_tb, self.pk, spark_session=self.spark)

In [3]:
%run ./etl_constants.ipynb

In [4]:
etl = UsersBronzeETL(spark, DT_START, DT_END)
df = etl.extract(source_tb=TARGET_RAW_TB)
etl.load(df, target_tb=TARGET_BRONZE_TB, drop=DROP_BRONZE)

--------------------
source_tb: oppai_challenge.patreon_users_raw


23/11/16 15:19:40 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/16 15:19:40 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/16 15:19:42 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/11/16 15:19:42 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore ahow@127.0.1.1
23/11/16 15:19:42 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


--------------------
--------------------
target_tb: oppai_challenge.patreon_users_bronze
drop: False


23/11/16 15:19:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


--------------------


In [5]:
df.count()

55566

In [9]:
spark.sql(f"SELECT * FROM {TARGET_BRONZE_TB}").count()

55566